# Laboratorio no calificado: Análisis de modelos TensorFlow

En los sistemas de producción, la decisión de desplegar un modelo suele ir más allá de las métricas globales (por ejemplo, la precisión) establecidas durante el entrenamiento. También es importante evaluar el rendimiento del modelo en diferentes escenarios. Por ejemplo, ¿su modelo de previsión meteorológica funciona igual de bien en verano que en invierno? ¿O su detector de defectos basado en cámaras sólo funciona en determinadas condiciones de iluminación? Este tipo de investigación ayuda a garantizar que su modelo puede manejar distintos casos. Más que eso, puede ayudar a descubrir cualquier sesgo aprendido que pueda resultar en una experiencia negativa para sus usuarios. Por ejemplo, si se supone que tienes una aplicación de género neutro, no querrás que tu modelo sólo funcione bien para uno y mal para otro.

En este laboratorio, estarás trabajando con [TensorFlow Model Analysis (TFMA)](https://www.tensorflow.org/tfx/guide/tfma) -- una librería construida específicamente para analizar el rendimiento de un modelo a través de diferentes configuraciones. Te permite especificar porciones de tus datos, luego calculará y visualizará cómo se desempeña tu modelo en cada porción. También puede establecer umbrales que su modelo debe cumplir antes de que se marque como listo para su despliegue. Esto le ayudará a tomar mejores decisiones sobre cualquier mejora que desee realizar para aumentar el rendimiento de su modelo y garantizar la equidad.

En este ejercicio, utilizará TFMA para analizar modelos entrenados en el conjunto de datos [Census Income dataset](http://archive.ics.uci.edu/ml/datasets/Census+Income). En concreto, deberá:

* estudiará y configurará los archivos de inicio para utilizarlos con TFMA
* crear un archivo de configuración para indicar a TFMA los datos que analizará y las métricas que calculará
* Visualizar los resultados de TFMA en un entorno de cuaderno.
* generar una serie temporal del rendimiento de un modelo
* Comparar el rendimiento de dos modelos para decidir cuál poner en producción.

*Créditos: Parte del código y las discusiones se basan en el [tutorial oficial] del equipo TensorFlow (https://www.tensorflow.org/tfx/tutorials/model_analysis/tfma_basic).

## Setup

En esta sección, primero configurará su espacio de trabajo para tener todos los módulos y archivos para trabajar con TFMA. Deberás 
* Instalar las bibliotecas necesarias, 
* Descargará los archivos de inicio que contendrán el conjunto de datos, el esquema y los modelos preentrenados que analizará.
* Preparar el conjunto de datos para que pueda ser consumido por TFMA.
* Observará cómo los modelos transforman las características en bruto.

### Instalar extensiones Jupyter
Si se ejecuta en un cuaderno Jupyter local, entonces estas extensiones Jupyter deben ser instaladas en el entorno antes de ejecutar Jupyter. Éstas ya están disponibles en Colab, por lo que dejaremos aquí los comandos como referencia.

```bash
jupyter nbextension enable --py widgetsnbextension --sys-prefix 
jupyter nbextension install --py --symlink tensorflow_model_analysis --sys-prefix 
jupyter nbextension enable --py tensorflow_model_analysis --sys-prefix 
```

### Instalar bibliotecas

Esto instalará todas las dependencias y tardará entre 6 y 8 minutos en completarse.


In [ ]:
# Upgrade pip to the latest version and install required packages
!pip install -U pip
!pip install --use-deprecated=legacy-resolver tensorflow_data_validation==1.1.0
!pip install --use-deprecated=legacy-resolver tensorflow-transform==1.0.0
!pip install --use-deprecated=legacy-resolver tensorflow-model-analysis==0.32.0
!pip install apache-beam==2.32.0

*Nota: En Google Colab, es necesario reiniciar el tiempo de ejecución en este punto para finalizar la actualización de los paquetes que acaba de instalar. **Por favor, no continúe con la siguiente sección sin reiniciar.** También puede ignorar los errores sobre la incompatibilidad de versiones de algunos de los paquetes incluidos, ya que no vamos a utilizarlos en este cuaderno.

### Comprobar la instalación

Ejecutando el código de abajo debería mostrar las versiones de los paquetes. Por favor, vuelve a ejecutar la instalación si ves errores y no olvides reiniciar el runtime después de la reinstalación.

In [ ]:
# Import packages and print versions
import tensorflow as tf
import tensorflow_model_analysis as tfma
import tensorflow_data_validation as tfdv

print('TF version: {}'.format(tf.__version__))
print('TFMA version: {}'.format(tfma.__version__))
print('TFDV version: {}'.format(tfdv.__version__))

### Cargar los archivos

A continuación, descargará los archivos que necesitará para este ejercicio:

* Conjuntos de datos de prueba
* Esquema de datos
* Modelos preentrenados

También hemos definido algunas variables globales a continuación para que puedas acceder a estos archivos a lo largo del cuaderno más fácilmente.

In [ ]:
import os

# String variables for file and directory names
URL = 'https://storage.googleapis.com/mlep-public/course_3/week4/C3_W4_Lab_1_starter_files.tar.gz'
TAR_NAME = 'C3_W4_Lab_1_starter_files.tar.gz'
BASE_DIR = 'starter_files'
DATA_DIR = os.path.join(BASE_DIR, 'data')
CSV_DIR = os.path.join(DATA_DIR, 'csv')
TFRECORD_DIR = os.path.join(DATA_DIR, 'tfrecord')
MODELS_DIR = os.path.join(BASE_DIR, 'models')
SCHEMA_FILE = os.path.join(BASE_DIR, 'schema.pbtxt')

In [ ]:
# descomente esta línea si ha descargado los archivos antes y desea restablecerlos
# !rm -rf {BASE_DIR}

# Download the tar file from GCP
!wget {URL}

# Extract the tar file to the base directory
!tar xzf {TAR_NAME}

# Delete tar file
!rm {TAR_NAME}

Puede ver los archivos y directorios de nivel superior ejecutando la celda que aparece a continuación (o simplemente utilizando el explorador de archivos situado en la parte izquierda de este Colab). Discutiremos lo que contiene cada uno en las siguientes secciones.

In [ ]:
print("Here's what we downloaded:")
!ls {BASE_DIR}

### Vista previa del conjunto de datos

El directorio `data/csv` contiene la división de prueba del conjunto de datos Census Income. Lo hemos dividido en varios archivos para este cuaderno de demostración:

* Datos_prueba.csv: 15.000 filas de datos de prueba.
* datos_prueba_1.csv` - primeras 5000 filas de datos_prueba.csv
* datos_prueba_2.csv` - las siguientes 5000 filas de datos_prueba.csv
* Datos_prueba_3.csv - últimas 5000 filas de Datos_prueba.csv

Puede ver la descripción de cada columna [aquí](http://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.names) (por favor, abra el enlace en una ventana nueva si Colab impide la descarga). Además, para simplificar, ya hemos preprocesado la columna `label` como binaria (es decir, `0` o `1`) para que coincida con la salida del modelo. En sus propios proyectos, las etiquetas pueden ser de un tipo de datos diferente (por ejemplo, cadena) y desea transformar primero para que pueda evaluar su modelo correctamente. Puede previsualizar las primeras filas a continuación:

In [ ]:
# Path to the full test set
TEST_DATA_PATH = os.path.join(CSV_DIR, 'data_test.csv')

# Preview the first few rows
!head {TEST_DATA_PATH}

### Parsear el Esquema

También ha descargado un esquema generado por [TensorFlow Data Validation](https://www.tensorflow.org/tfx/data_validation/). Ya deberías estar familiarizado con este tipo de archivo de cursos anteriores. Lo cargarás ahora para poder utilizarlo en las partes posteriores del cuaderno.

In [ ]:
# Load the schema as a protocol buffer
SCHEMA = tfdv.load_schema_text(SCHEMA_FILE)

# Display the schema
tfdv.display_schema(SCHEMA)

### Utilizar el esquema para crear TFRecords

TFMA necesita un archivo TFRecord de entrada, por lo que necesitas convertir los CSVs en el directorio de datos. Si has hecho los laboratorios anteriores, sabrás que esto se puede hacer fácilmente con `ExampleGen`. Para este cuaderno, sin embargo, se utilizará la función de ayuda a continuación para demostrar cómo se puede hacer fuera de una tubería TFX. Pasarás el esquema que cargaste en el paso anterior para determinar el tipo correcto de cada característica.

In [ ]:
# imports for helper function
import csv
from tensorflow.core.example import example_pb2
from tensorflow_metadata.proto.v0 import schema_pb2

def csv_to_tfrecord(schema, csv_file, tfrecord_file):
  ''' Convierte un archivo csv en un tfrecord
  Args:
    schema (schema_pb2) - protobuf de esquema de TFDV
    csv_file (cadena) - archivo a convertir en tfrecord
    tfrecord_file (cadena) - nombre del archivo tfrecord a crear

  Devuelve
    nombre de archivo de tfrecord
  '''

  # Abrir archivo CSV para su lectura. Cada fila se mapea como un diccionario.
  reader = csv.DictReader(open(csv_file, 'r'))
  
  # Inicializar la lista de ejemplos de TF
  examples = []

  # Para cada fila en CSV, crear un Ejemplo TF basado en
  # el esquema y añádalo a la lista
  for line in reader:

    # Intialize example
    example = example_pb2.Example()

    # Bucle a través de características en el esquema
    for feature in schema.feature:

      # Obtener el nombre de la característica actual
      key = feature.name

      # Rellenar valores según el tipo de datos de la entidad actual
      if feature.type == schema_pb2.FLOAT:
        example.features.feature[key].float_list.value[:] = (
            [float(line[key])] if len(line[key]) > 0 else [])
      elif feature.type == schema_pb2.INT:
        example.features.feature[key].int64_list.value[:] = (
            [int(line[key])] if len(line[key]) > 0 else [])
      elif feature.type == schema_pb2.BYTES:
        example.features.feature[key].bytes_list.value[:] = (
            [line[key].encode('utf8')] if len(line[key]) > 0 else [])
        
    # Append to the list
    examples.append(example)

  # Escribir ejemplos en el archivo tfrecord
  with tf.io.TFRecordWriter(tfrecord_file) as writer:
    for example in examples:
      writer.write(example.SerializeToString())
  
  return tfrecord_file

El siguiente código hará la conversión y hemos definido algunas variables globales más que usará en ejercicios posteriores.

In [ ]:
# Create tfrecord directory
!mkdir {TFRECORD_DIR}

# Create list of tfrecord files
tfrecord_files = [csv_to_tfrecord(SCHEMA, f'{CSV_DIR}/{name}', f"{TFRECORD_DIR}/{name.replace('csv','tfrecord')}") 
  for name in os.listdir(CSV_DIR)]

# Print created files
print(f'files created: {tfrecord_files}')

# Create variables for each tfrecord
TFRECORD_FULL = os.path.join(TFRECORD_DIR, 'data_test.tfrecord')
TFRECORD_DAY1 = os.path.join(TFRECORD_DIR, 'data_test_1.tfrecord')
TFRECORD_DAY2 = os.path.join(TFRECORD_DIR, 'data_test_2.tfrecord')
TFRECORD_DAY3 = os.path.join(TFRECORD_DIR, 'data_test_3.tfrecord')

# Delete unneeded variable
del tfrecord_files

### Modelos preentrenados

Por último, también has descargado modelos Keras preentrenados y están almacenados en el directorio `models/`. TFMA soporta diferentes tipos de modelos, incluyendo modelos TF Keras, modelos basados en APIs genéricas de firmas TF2, así como modelos basados en estimadores TF. La guía [get_started](https://www.tensorflow.org/tfx/model_analysis/get_started) contiene la lista completa de tipos de modelos soportados y sus restricciones. También puede consultar las [FAQ](https://www.tensorflow.org/tfx/model_analysis/faq) para ver ejemplos sobre cómo configurar estos modelos.

Hemos incluido tres modelos y puede elegir analizar cualquiera de ellos en las secciones posteriores. Se han guardado en formato [SavedModel](https://www.tensorflow.org/guide/saved_model) que es el predeterminado cuando se guarda con la API de modelos de Keras.

In [ ]:
# list model directories
!ls {MODELS_DIR}

# Create string variables for each model directory
MODEL1_FILE = os.path.join(MODELS_DIR, 'model1')
MODEL2_FILE = os.path.join(MODELS_DIR, 'model2')
MODEL3_FILE = os.path.join(MODELS_DIR, 'model3')

Como ya se ha mencionado, estos modelos se entrenaron con el conjunto de datos [Census Income dataset](http://archive.ics.uci.edu/ml/datasets/Census+Income). La etiqueta es `1` si una persona gana más de 50.000 USD y `0` si gana menos o igual. Puede cargar uno de los modelos y consultar el resumen para hacerse una idea de su arquitectura. Los tres modelos utilizan la misma arquitectura, pero se entrenaron con diferentes épocas para simular un rendimiento variable.

In [ ]:
# Load model 1
model = tf.keras.models.load_model(MODEL1_FILE)

# Print summary. You can ignore the warnings at the start.
model.summary()

Puedes ver el código para construir estos en el próximo laboratorio. Por ahora, sólo tendrá que tomar nota de algunas cosas. En primer lugar, la salida es una sola unidad densa con una activación sigmoide (es decir, `denso_5` arriba). Esto es estándar para problemas de clasificación binaria.

Otra es que el modelo se exporta con una capa de transformación. Esto se puede ver en el resumen de arriba en la fila inferior llamada `transform_features_layer` y no está conectada a las otras capas. De laboratorios anteriores, usted sabrá que esto se toma de la gráfica generada por el componente `Transform`. Ayuda a evitar sesgos en el entrenamiento asegurándose de que las entradas brutas se transforman de la misma manera que el modelo espera. También está disponible como propiedad `tft_layer` del objeto modelo.

In [ ]:
# Se puede acceder a la capa de transformación de dos maneras. Estos son equivalentes.
model.get_layer('transform_features_layer') is model.tft_layer

TFMA invoca esta capa automáticamente para sus entradas brutas, pero hemos incluido un breve fragmento a continuación para demostrar cómo funciona la transformación. Puede ver que las características brutas se reformatean para que sean aceptables para el modelo. Las características numéricas en bruto se escalan y las características categóricas en bruto (cadena) se codifican en vectores de un solo golpe.

In [ ]:
from tensorflow_transform.tf_metadata import schema_utils

# Load one tfrecord
tfrecord_file = tf.data.TFRecordDataset(TFRECORD_DAY1)

# Parse schema object as a feature spec
feature_spec = schema_utils.schema_as_feature_spec(SCHEMA).feature_spec

# Create a batch from the dataset
for records in tfrecord_file.batch(1).take(1):

  # Parse the batch to get a dictionary of raw features
  parsed_examples = tf.io.parse_example(records, feature_spec)

  # Print the results
  print("\nRAW FEATURES:")
  for key, value in parsed_examples.items():
    print(f'{key}: {value.numpy()}')
  
  # Pop the label since the model does not expect a label input
  parsed_examples.pop('label')

  # Transform the rest of the raw features using the transform layer
  transformed_examples = model.tft_layer(parsed_examples)

  # Print the input to the model
  print("\nTRANSFORMED FEATURES:")
  for key, value in transformed_examples.items():
    print(f'{key}: {value.numpy()}')

Las características transformadas pueden introducirse en el modelo para obtener las predicciones. El siguiente fragmento de código lo demuestra y utilizamos una métrica de bajo umbral [BinaryAccuracy](https://www.tensorflow.org/api_docs/python/tf/keras/metrics/BinaryAccuracy) para comparar las etiquetas verdaderas y las predicciones del modelo.

In [ ]:
from tensorflow_transform.tf_metadata import schema_utils

# Load one tfrecord
tfrecord_file = tf.data.TFRecordDataset(TFRECORD_DAY1)

# Parse schema object as a feature spec
feature_spec = schema_utils.schema_as_feature_spec(SCHEMA).feature_spec

# Create a batch from the dataset
for records in tfrecord_file.batch(5).take(1):

  # Get the label values from the raw input
  parsed_examples = tf.io.parse_example(records, feature_spec)
  y_true = parsed_examples.pop('label')
  print(f'labels:\n {y_true.numpy()}\n')
  
  # Transform the raw features and pass to the model to get predictions
  transformed_examples = model.tft_layer(parsed_examples)
  y_pred = model(transformed_examples)
  print(f'predictions:\n {y_pred.numpy()}\n')
  
  # Measure the binary accuracy
  metric = tf.keras.metrics.BinaryAccuracy(threshold=0.3)
  metric.update_state(y_true, y_pred)
  print(f'binary accuracy: {metric.result().numpy()}\n')

La última cosa a tener en cuenta es que el modelo también se exporta con una [serving signature](https://www.tensorflow.org/guide/saved_model#specifying_signatures_during_export). Usted sabrá más acerca de esto en el próximo laboratorio y en las partes posteriores de la especialización, pero por ahora, usted puede pensar en ello como una configuración para cuando el modelo se despliega para la inferencia. Para este modelo en particular, la firma por defecto está configurado para transformar los lotes de características en bruto serializados antes de alimentar a la entrada del modelo. De esta manera, usted no tendría que codificar explícitamente las transformaciones como se muestra en el fragmento anterior. Usted puede simplemente pasar en lotes de datos directamente como se muestra a continuación.

In [ ]:
# Load one tfrecord
tfrecord_file = tf.data.TFRecordDataset(TFRECORD_DAY1)

# Print available signatures
print(f'model signatures: {model.signatures}\n')

# Create a batch
for records in tfrecord_file.batch(5).take(1):

  # Pass the batch to the model serving signature to get predictions
  output = model.signatures['serving_default'](examples=records)

  # Print results
  print(f"predictions:\n {output['output_0']}\n")

TFMA accede a esta firma de modelo para poder trabajar con los datos brutos y evaluar el modelo para obtener las métricas. No sólo eso, también puede extraer características específicas y valores de dominio de su conjunto de datos antes de calcular estas métricas. Veamos cómo se hace esto en la siguiente sección.

## Configurar y ejecutar TFMA

Con el conjunto de datos y el modelo ya disponibles, puede pasar a utilizar TFMA. Hay algunos pasos adicionales necesarios:

* Crear un mensaje de protocolo `tfma.EvalConfig` que contenga detalles sobre los modelos, métricas y datos que desea analizar.
* Crear un `tfma.EvalSharedModel` que apunte a los modelos guardados.
* Especifique una ruta de salida donde se almacenarán los resultados del análisis. 

###Crear EvalConfig

El [tfma.EvalConfig()](https://www.tensorflow.org/tfx/model_analysis/api_docs/python/tfma/EvalConfig?hl=nn&skip_cache=true) es un mensaje de protocolo que configura el análisis. Aquí se especifica:

* [`model_specs`](https://www.tensorflow.org/tfx/model_analysis/api_docs/python/tfma/ModelSpec) - mensaje que contiene al menos la clave de la etiqueta para que pueda extraerse de los datos de la evaluación/prueba.

* [`metrics_specs`](https://www.tensorflow.org/tfx/model_analysis/api_docs/python/tfma/MetricsSpec) - que contiene las métricas que desea evaluar. Puede encontrar una guía completa [aquí](https://www.tensorflow.org/tfx/model_analysis/metrics) y utilizará las métricas de clasificación binaria para este ejercicio.

* [`slicing_specs`](https://www.tensorflow.org/tfx/model_analysis/api_docs/python/tfma/SlicingSpec) - contiene los cortes de características para los que desea calcular las métricas. Una breve descripción de los diferentes tipos de cortes se muestra [aquí](https://www.tensorflow.org/tfx/model_analysis/setup#slicing_specs)

La configuración de la evaluación debe pasarse como un mensaje de protocolo y puedes utilizar el módulo [google.protobuf.text_format](https://googleapis.dev/python/protobuf/latest/google/protobuf/text_format.html) para ello, como se muestra a continuación.

In [ ]:
import tensorflow_model_analysis as tfma
from google.protobuf import text_format

# Setup tfma.EvalConfig settings
eval_config = text_format.Parse("""
  ## Model information
  model_specs {
    # For keras (and serving models), you need to add a `label_key`.
    label_key: "label"
  }

  ## Post training metric information. These will be merged with any built-in
  ## metrics from training.
  metrics_specs {
    metrics { class_name: "ExampleCount" }
    metrics { class_name: "BinaryAccuracy" }
    metrics { class_name: "BinaryCrossentropy" }
    metrics { class_name: "AUC" }
    metrics { class_name: "AUCPrecisionRecall" }
    metrics { class_name: "Precision" }
    metrics { class_name: "Recall" }
    metrics { class_name: "MeanLabel" }
    metrics { class_name: "MeanPrediction" }
    metrics { class_name: "Calibration" }
    metrics { class_name: "CalibrationPlot" }
    metrics { class_name: "ConfusionMatrixPlot" }
    # ... add additional metrics and plots ...
  }

  ## Slicing information

  # overall slice
  slicing_specs {}

  # slice specific features
  slicing_specs {
    feature_keys: ["sex"]
  }
  slicing_specs {
    feature_keys: ["race"]
  }

  # slice specific values from features
  slicing_specs {
    feature_values: {
      key: "native-country"
      value: "Cambodia"
    }
  }
  slicing_specs {
    feature_values: {
      key: "native-country"
      value: "Canada"
    }
  }

  # slice feature crosses
  slicing_specs {
    feature_keys: ["sex", "race"]
  }
""", tfma.EvalConfig())

### Crear EvalSharedModel

TFMA también requiere una instancia [EvalSharedModel](https://www.tensorflow.org/tfx/model_analysis/api_docs/python/tfma/types/EvalSharedModel) que apunte a tu modelo para que pueda ser compartido entre múltiples hilos en el mismo proceso. Esta instancia incluye información sobre el tipo de modelo (keras, etc) y cómo cargar y configurar el modelo desde su ubicación guardada en disco (por ejemplo, etiquetas, etc). La API [tfma.default_eval_shared_model()](https://www.tensorflow.org/tfx/model_analysis/api_docs/python/tfma/default_eval_shared_model) se puede utilizar para crear esto dada la ubicación del modelo y la configuración de la evaluación.

In [ ]:
# Create a tfma.EvalSharedModel that points to your model.
# You can ignore the warnings generated.
eval_shared_model = tfma.default_eval_shared_model(
    eval_saved_model_path=MODEL1_FILE,
    eval_config=eval_config)

In [ ]:
# Show properties of EvalSharedModel
print(f'EvalSharedModel contents: {eval_shared_model}')

### Ejecutar TFMA

Con la configuración completa, sólo tiene que declarar un directorio de salida y luego ejecutar TFMA. Pasará la configuración de evaluación, el modelo compartido, el conjunto de datos y el directorio de salida a [`tfma.run_model_analysis()`](https://www.tensorflow.org/tfx/model_analysis/api_docs/python/tfma/run_model_analysis) como se muestra a continuación. Esto creará un [`tfma.EvalResult`](https://www.tensorflow.org/tfx/model_analysis/api_docs/python/tfma/EvalResult) que puede utilizar más tarde para la representación de métricas y gráficos.

In [ ]:
# Specify output path for the evaluation results
OUTPUT_DIR = os.path.join(BASE_DIR, 'output')

# Run TFMA. You can ignore the warnings generated.
eval_result = tfma.run_model_analysis(
    eval_shared_model=eval_shared_model,
    eval_config=eval_config,
    data_location=TFRECORD_FULL,
    output_path=OUTPUT_DIR)

## Visualización de métricas y gráficos

También puede visualizar los resultados utilizando los métodos TFMA. En esta sección, verá las métricas y gráficos devueltos para los diferentes cortes especificados en la configuración de evaluación.

### Renderización de métricas

Puede ver las métricas con el método [`tfma.view.render_slicing_metrics()`](https://www.tensorflow.org/tfx/model_analysis/api_docs/python/tfma/view/render_slicing_metrics). Por defecto, las vistas mostrarán el corte `Overall`. Para ver una porción en particular puede pasar un nombre de característica al argumento `slicing_column` como se muestra a continuación. Puede visualizar las diferentes métricas a través del menú desplegable `Show` y puede pasar el ratón por encima de los gráficos de barras para mostrar el valor exacto medido. 

Le animamos a que pruebe las distintas opciones que vea y también a que modifique el comando. He aquí algunos ejemplos:

* Quitando el argumento `slicing_column` obtendrá el corte global.
* También puede pasar `race` (ya que se especificó en la configuración de evaluación) para ver los resultados de ese corte en particular.
* Usando el deslizador `Examples (Weighted) Threshold` por encima de 5421 eliminará el corte `Female` porque tiene menos ejemplos que ese.
* Cambiando el desplegable "Ver" a "Histograma de métricas" mostrará los resultados divididos en cubos. Por ejemplo, si su columna de corte es "sexo" y el desplegable "Tipo de histograma" está en "Recuentos de cortes", entonces tendrá un corte en dos de los 10 cubos (por defecto), ya que sólo tenemos dos valores para esa característica ("Masculino" y "Femenino"). El eje x muestra los valores de la métrica en el desplegable "Seleccionar métrica". Esta es la vista por defecto cuando el número de cortes es grande.
* En la parte inferior de la pantalla, observará que las medidas también se presentan en formato tabular. Puede ordenarlas haciendo clic en los encabezados de los nombres de las características.

In [ ]:
# Render metrics for a feature
tfma.view.render_slicing_metrics(eval_result, slicing_column='sex')

### Más cortes

Si aún no lo has hecho, también puedes pasar el valor `native-country` a la columna de segmentación. La diferencia en esta visualización es que antes sólo especificamos dos de sus valores en la configuración de evaluación. Esto es útil si sólo desea estudiar un subgrupo de una característica en particular y no todo el dominio.

In [ ]:
# Render metrics for feature. Review EvalConfig message to see what values were selected.
tfma.view.render_slicing_metrics(eval_result, slicing_column='native-country')

TFMA también permite crear cruces de características para analizar combinaciones de características. Nuestra configuración original creó un cruce entre `sex` y `race` y puede pasarlo como [SlicingSpec](https://www.tensorflow.org/tfx/model_analysis/api_docs/python/tfma/SlicingSpec) como se muestra a continuación.

In [ ]:
# Render metrics for feature crosses
tfma.view.render_slicing_metrics(
    eval_result,
    slicing_spec=tfma.SlicingSpec(
        feature_keys=['sex', 'race']))

En algunos casos, al cruzar las dos columnas se crean muchas combinaciones. Puede acotar los resultados para fijarse sólo en valores específicos especificándolo en el argumento `slicing_spec`. A continuación se muestran los resultados de la característica `sex` para la raza `Other`.

In [ ]:
# Narrow down the feature crosses by specifying feature values
tfma.view.render_slicing_metrics(
    eval_result,
    slicing_spec=tfma.SlicingSpec(
        feature_keys=['sex'], feature_values={'race': 'Other'}))

### Representación de gráficos

Cualquier gráfico que se haya añadido a `tfma.EvalConfig` como `metric_specs` posterior al entrenamiento puede visualizarse utilizando [`tfma.view.render_plot`](https://www.tensorflow.org/tfx/model_analysis/api_docs/python/tfma/view/render_plot).

Al igual que con las métricas, los gráficos pueden visualizarse por cortes. A diferencia de las métricas, sólo pueden mostrarse los gráficos de un valor de corte concreto, por lo que debe utilizarse `tfma.SlicingSpec`, que debe especificar tanto el nombre como el valor de la característica de corte. Si no se especifica ninguna porción, se utilizarán los gráficos de la porción `Overall`.

El siguiente ejemplo muestra los gráficos calculados para el corte `sex:Male`. Puede hacer clic en los nombres de la parte inferior del gráfico para ver un tipo de gráfico diferente. Alternativamente, puede marcar la casilla `Mostrar todos los gráficos` para mostrar todos los gráficos en una pantalla.

In [ ]:
# Render plots
tfma.view.render_plot(
    eval_result,
    tfma.SlicingSpec(feature_values={'sex': 'Male'}))

## Seguimiento del rendimiento del modelo a lo largo del tiempo

Su conjunto de datos de entrenamiento se utilizará para entrenar su modelo, y es de esperar que sea representativo de su conjunto de datos de prueba y de los datos que se enviarán a su modelo en producción.  Sin embargo, aunque los datos en las solicitudes de inferencia pueden seguir siendo los mismos que sus datos de entrenamiento, también pueden empezar a cambiar lo suficiente como para que cambie el rendimiento de su modelo. Esto significa que necesita supervisar y medir el rendimiento de su modelo de forma continua para que pueda ser consciente de los cambios y reaccionar ante ellos.

Veamos cómo puede ayudarle TFMA. Cargará tres conjuntos de datos diferentes y comparará los resultados del análisis del modelo utilizando el método [`render_time_series()`](https://www.tensorflow.org/tfx/model_analysis/api_docs/python/tfma/view/render_time_series).

In [ ]:
import os

# Put data paths we prepared earlier in a list
TFRECORDS = [TFRECORD_DAY1, TFRECORD_DAY2, TFRECORD_DAY3]

# Initialize output paths list for each result 
output_paths = []

# Run eval on each tfrecord separately
for num, tfrecord in enumerate(TFRECORDS):

  # Use the same model as before
  eval_shared_model = tfma.default_eval_shared_model(
      eval_saved_model_path=MODEL1_FILE,
      eval_config=eval_config)

  # Prepare output path name
  output_path = os.path.join('.', 'time_series', str(num))
  output_paths.append(output_path)

  # Run TFMA on the current tfrecord in the loop
  tfma.run_model_analysis(eval_shared_model=eval_shared_model,
                          eval_config=eval_config,
                          data_location=tfrecord,
                          output_path=output_path)

En primer lugar, imagina que has entrenado y desplegado tu modelo ayer. Y ahora quiere ver cómo funciona con los nuevos datos que llegan hoy.  La visualización comenzará mostrando el AUC. Desde la interfaz de usuario, puede:

* Añadir otras métricas utilizando el menú "Añadir serie de métricas".
* Cerrar los gráficos no deseados haciendo clic en x
* Pasar el ratón por encima de los puntos de datos (los extremos de los segmentos de línea en el gráfico) para obtener más detalles.

Nota: En los gráficos de series métricas, el eje x es simplemente el nombre del directorio del modelo que estás examinando.

In [ ]:
# Load results for day 1 and day 2 datasets
eval_results_from_disk = tfma.load_eval_results(output_paths[:2])

# Visualize results
tfma.view.render_time_series(eval_results_from_disk)

Ahora imagina que ha pasado otro día y quieres ver cómo le va con los nuevos datos que llegan hoy.

In [ ]:
# Load results for all three days
eval_results_from_disk = tfma.load_eval_results(output_paths)

# Visualize the results
tfma.view.render_time_series(eval_results_from_disk)

Este tipo de investigación le permite ver si su modelo se comporta mal con los nuevos datos. A partir de estos resultados, puede tomar la decisión de reentrenar su modelo de producción. Es posible que el reentrenamiento no siempre produzca los mejores resultados, por lo que también necesita una forma de detectarlo. Verá cómo TFMA le ayuda en este sentido en la siguiente sección.

## Validación de modelos

TFMA puede configurarse para evaluar varios modelos al mismo tiempo. Típicamente, esto se hace para comparar un modelo candidato contra una línea de base (como el modelo actualmente en servicio) para determinar cuáles son las diferencias de rendimiento en las métricas. Cuando se configuran [umbrales](https://www.tensorflow.org/tfx/model_analysis/api_docs/python/tfma/MetricThreshold), TFMA producirá un registro [`tfma.ValidationResult`](https://www.tensorflow.org/tfx/model_analysis/api_docs/python/tfma/ValidationResult) indicando si el rendimiento coincide con las expectativas.

A continuación, reconfigurará los ajustes de EvalConfig para comparar dos modelos: un candidato y una línea de base. También validará el rendimiento del candidato frente a la línea de base estableciendo un [`tmfa.MetricThreshold`](https://www.tensorflow.org/tfx/model_analysis/api_docs/python/tfma/MetricThreshold) en la métrica `BinaryAccuracy`. Esto ayuda a determinar si el nuevo modelo puede reemplazar al modelo actual.

In [ ]:
# Setup tfma.EvalConfig setting with metric thresholds
eval_config_with_thresholds = text_format.Parse("""
  ## Model information
  model_specs {
    name: "candidate"
    label_key: "label"
  }
  model_specs {
    name: "baseline"
    label_key: "label"
    is_baseline: true
  }

  ## Post training metric information
  metrics_specs {
    metrics { class_name: "ExampleCount" }
    metrics {
      class_name: "BinaryAccuracy"
      threshold {
        # Ensure that metric is always > 0.9
        value_threshold {
          lower_bound { value: 0.9 }
        }
        # Ensure that metric does not drop by more than a small epsilon
        # e.g. (candidate - baseline) > -1e-10 or candidate > baseline - 1e-10
        change_threshold {
          direction: HIGHER_IS_BETTER
          absolute { value: -1e-10 }
        }
      }
    }
    metrics { class_name: "BinaryCrossentropy" }
    metrics { class_name: "AUC" }
    metrics { class_name: "AUCPrecisionRecall" }
    metrics { class_name: "Precision" }
    metrics { class_name: "Recall" }
    metrics { class_name: "MeanLabel" }
    metrics { class_name: "MeanPrediction" }
    metrics { class_name: "Calibration" }
    metrics { class_name: "CalibrationPlot" }
    metrics { class_name: "ConfusionMatrixPlot" }
    # ... add additional metrics and plots ...
  }

  ## Slicing information
  slicing_specs {}  # overall slice
  slicing_specs {
    feature_keys: ["race"]
  }
  slicing_specs {
    feature_keys: ["sex"]
  }
""", tfma.EvalConfig())

# Create tfma.EvalSharedModels that points to the candidate and baseline
candidate_model_path = MODEL1_FILE
baseline_model_path = MODEL2_FILE

eval_shared_models = [
  tfma.default_eval_shared_model(
      model_name=tfma.CANDIDATE_KEY,
      eval_saved_model_path=candidate_model_path,
      eval_config=eval_config_with_thresholds),
  tfma.default_eval_shared_model(
      model_name=tfma.BASELINE_KEY,
      eval_saved_model_path=baseline_model_path,
      eval_config=eval_config_with_thresholds),
]

# Specify validation path
validation_output_path = os.path.join(OUTPUT_DIR, 'validation')

# Run TFMA on the two models
eval_result_with_validation = tfma.run_model_analysis(
    eval_shared_models,
    eval_config=eval_config_with_thresholds,
    data_location=TFRECORD_FULL,
    output_path=validation_output_path)

Cuando se ejecutan evaluaciones con uno o más modelos frente a una línea de base, TFMA añade automáticamente métricas diferentes para todas las métricas calculadas durante la evaluación. Estas métricas reciben el mismo nombre que la métrica correspondiente, pero con la cadena `_diff` añadida al nombre de la métrica. Un valor positivo de estas métricas `_diff` indica una mejora del rendimiento con respecto a la línea de base.

Al igual que en la sección anterior, puede ver los resultados con `render_time_series()`.

In [ ]:
# Render results
tfma.view.render_time_series(eval_result_with_validation)

Puede utilizar [`tfma.load_validator_result`](https://www.tensorflow.org/tfx/model_analysis/api_docs/python/tfma/load_validation_result) para ver los resultados de la validación que especificó con la configuración del umbral. En este ejemplo, la validación falla porque `BinaryAccuracy` está por debajo del umbral.

In [ ]:
# Print validation result
validation_result = tfma.load_validation_result(validation_output_path)
print(validation_result.validation_ok)

**¡Enhorabuena! Ya ha explorado los diferentes métodos de análisis de modelos utilizando TFMA. En la siguiente sección, verá cómo estos pueden encajar en una canalización TFX para que pueda automatizar el proceso y almacenar los resultados en su directorio de canalización y almacén de metadatos.**